<a href="https://colab.research.google.com/github/M-Awwab-Khan/most-comprehensive-movies-analysis/blob/main/Movie_Metrics_An_Analyst_Approach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install -q kaggle

In [2]:
from google.colab import userdata
import os

os.environ["KAGGLE_KEY"] = userdata.get('key')
os.environ["KAGGLE_USERNAME"] = userdata.get('username')

In [3]:
!kaggle datasets download -d rounakbanik/the-movies-dataset

 99% 225M/228M [00:08<00:00, 32.5MB/s]
100% 228M/228M [00:08<00:00, 27.9MB/s]


In [4]:
! unzip "the-movies-dataset.zip"

Archive:  the-movies-dataset.zip
  inflating: credits.csv             
  inflating: keywords.csv            
  inflating: links.csv               
  inflating: links_small.csv         
  inflating: movies_metadata.csv     
  inflating: ratings.csv             
  inflating: ratings_small.csv       


In [5]:
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import ast
from wordcloud import WordCloud, STOPWORDS
import plotly
import plotly.offline as py
import plotly.graph_objs as go
import plotly.tools as tls
import warnings
warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
sns.set(font_scale=1.25)
pd.set_option('display.max_colwidth', 50)
py.init_notebook_mode(connected=True)

In [6]:
df = pd.read_csv('/content/movies_metadata.csv')

In [7]:
df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


##Understanding the Dataset
The dataset above was obtained through the TMDB API. The movies available in this dataset are in correspondence with the movies that are listed in the MovieLens Latest Full Dataset comprising of 26 million ratings on 45,000 movies from 27,000 users. Let us have a look at the features that are available to us.

In [8]:
df.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

###Features
**adult:** Indicates if the movie is X-Rated or Adult.

**belongs_to_collection:** A stringified dictionary that gives information on the movie series the particular film belongs to.

**budget**: The budget of the movie in dollars.

**genres**: A stringified list of dictionaries that list out all the genres associated with the movie.

**homepage**: The Official Homepage of the move.

**id**: The ID of the move.

**imdb_id**: The IMDB ID of the movie.

**original_language**: The language in which the movie was originally
shot in.

**original_title**: The original title of the movie.

**overview**: A brief blurb of the movie.

**popularity**: The Popularity Score assigned by TMDB.

**poster_path**: The URL of the poster image.

**production_companies**: A stringified list of production companies involved with the making of the movie.

**production_countries**: A stringified list of countries where the movie was shot/produced in.

**release_date**: Theatrical Release Date of the movie.

**revenue**: The total revenue of the movie in dollars.

**runtime**: The runtime of the movie in minutes.

**spoken_languages**: A stringified list of spoken languages in the film.

**status**: The status of the movie (Released, To Be Released,
Announced, etc.)

**tagline**: The tagline of the movie.

**title**: The Official Title of the movie.

**video**: Indicates if there is a video present of the movie with
TMDB.

**vote_average**: The average rating of the movie.

**vote_count**: The number of votes by users, as counted by TMDB.

In [9]:
df.shape

(45466, 24)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

There are a total of 45,466 movies with 24 features. Most of the features have very few NaN values (apart from homepage and tagline). I will attempt at cleaning this dataset to a form suitable for analysis in the next section.

## Data Wrangling

The data that was originally obtained was in the form of a JSON File. This was converted manually into a CSV file that could be loaded into a Pandas DataFrame effortlessly. In other words, the dataset we have in our hands is already relatively clean.

Let us start by removing the features that are not useful to us.

In [21]:
df = df.drop(['imdb_id', 'adult'], axis=1)

In [13]:
df[df['original_title'] != df['title']][['title', 'original_title']].head()

,title,original_title
28,The City of Lost Children,La Cité des Enfants Perdus
29,Shanghai Triad,摇啊摇，摇到外婆桥
32,Wings of Courage,"Guillaumet, les ailes du courage"
57,The Postman,Il postino
58,The Confessional,Le confessionnal


The original title refers to the title of the movie in the native language in which the movie was shot. As such, I will prefer using the translated, and hence, will drop the original titles altogether. We will be able to deduce if the movie is a foreign language film by looking at the original_language feature so no tangible information is lost in doing so.

In [14]:
df = df.drop('original_title', axis=1)

In [15]:
df[df['revenue'] == 0].shape

(38052, 22)

We see that huge number of movies have 0 revenue, which doesn't make sense. However there are still 7000 movies with sensible revenue, hence we will replace these with NaN values to prevent them from disturbing the analysis.

In [16]:
df['revenue'] = df['revenue'].replace(0, np.nan)

The budget column is of object type which indicates, there might be some strange values in that column which prevents it from being converted. Similarly most of values in this column are also 0 which we will convert to NaN values to prevent them from disturbing the analysis.

In [18]:
df['budget'] = pd.to_numeric(df['budget'], errors='coerce')
df['budget'] = df['budget'].replace(0, np.nan)
df[df['budget'].isnull()].shape

(36576, 22)

To answer certain questions from this dataset, we need to obtain two columns namely,
- return
- year

The return feature is extremely insightful as it will give us a more accurate picture of the financial success of a movie. Presently, our data will not be able to judge if a \$200 million budget movie that earned \$100 million did better than a \$50,000 budget movie taking in \$200,000. This feature will be able to capture that information.

A return value > 1 would indicate profit whereas a return value < 1 would indicate a loss.

In [19]:
df['return'] = df['revenue'] / df['budget']
df[df['return'].isnull()].shape

(40085, 23)

We have approximately 5000 movies with return values, which is about 11% of entire dataset. Although this may seem small, but it is enough to perform some useful analysis and extract insights.

In [20]:
df['year'] = pd.to_datetime(df['release_date'], errors='coerce').dt.year

In [ ]:
df.drop